In [112]:
#Libraries

#Data Analysis Libs
import pandas as pd
import numpy as np

In [114]:
od_matrix = pd.read_csv('/local/tarciso/masters/data/bus_trips/latest/otp-od/2017_06_16_full_od_trips.csv', parse_dates=['start_time','end_time'])

In [115]:
od_matrix.head()

,cardNum,user_trip_id,itinerary_id,leg_id,route,busCode,tripNum,from_stop_id,start_time,from_stop_lat,from_stop_lon,to_stop_id,end_time,to_stop_lat,to_stop_lon,leg_duration
0,844324.0,4700,1,1,342,BA120,10.0,34915.0,2017-06-16 14:50:47,-25.396340,-49.202170,30448.0,2017-06-16 15:09:47,-25.393374,-49.240917,0 days 00:19:00.000000000
1,948116.0,6671,2,1,684,HA610,1.0,39372.0,2017-06-16 04:48:26,-25.593450,-49.333880,31053.0,2017-06-16 05:12:47,-25.513098,-49.295303,0 days 00:24:21.000000000
2,948116.0,6671,2,3,204,HL309,1.0,26252.0,2017-06-16 05:29:56,-25.512983,-49.294505,25917.0,2017-06-16 05:57:00,-25.426995,-49.264729,0 days 00:27:04.000000000
3,948116.0,6671,2,5,361,BC906,1.0,29889.0,2017-06-16 06:21:06,-25.426910,-49.263080,28979.0,2017-06-16 06:27:10,-25.417287,-49.249938,0 days 00:06:04.000000000
4,948116.0,6672,5,1,216,BA603,7.0,28979.0,2017-06-16 15:25:37,-25.417287,-49.249938,28332.0,2017-06-16 16:00:52,-25.476127,-49.292362,0 days 00:35:15.000000000


In [116]:
len(od_matrix)

133695

In [117]:
bus_trips_filepath = '/local/tarciso/masters/data/bus_trips/latest/enhanced-buste/2017_06_16_bus_trips.csv'
bus_trips = pd.read_csv(bus_trips_filepath, dtype={'route': object},parse_dates=['gps_datetime']) \
                .sort_values(['route','busCode','tripNum','gps_datetime']) \
                .assign(route = lambda x: x['route'].astype(str).str.replace("\.0",'').str.zfill(3)) \
                .rename(index=str, columns={'stopPointId':'stop_id'})

In [118]:
bus_trips.head()

,route,busCode,tripNum,stop_id,gps_datetime,stop_lat,stop_lon,parent_station
121790,001,BN998,1.0,29887,2017-06-16 07:01:26,-25.428171,-49.264558,NaN
121791,001,BN998,1.0,31453,2017-06-16 07:02:04,-25.430388,-49.263602,NaN
121792,001,BN998,1.0,31454,2017-06-16 07:02:38,-25.433503,-49.262257,NaN
121793,001,BN998,1.0,30748,2017-06-16 07:05:09,-25.435187,-49.264933,NaN
121794,001,BN998,1.0,30749,2017-06-16 07:05:22,-25.435959,-49.266889,NaN


In [119]:
boardings_df = od_matrix.filter(['route','busCode','tripNum','from_stop_id','start_time']) \
                        .groupby(['route','busCode','tripNum','from_stop_id']) \
                        .count() \
                        .reset_index() \
                        .rename(index=str, columns={'from_stop_id':'stop_id', 'start_time':'boarding_cnt'}) \
                        .sort_values('boarding_cnt', ascending=False)

In [120]:
boardings_df.head()

,route,busCode,tripNum,stop_id,boarding_cnt
37517,550,GE717,2.0,26298.0,341
30561,503,GE696,4.0,27560.0,257
30727,503,GR021,12.0,27560.0,216
30695,503,GR021,10.0,27560.0,194
32100,506,EL059,2.0,25830.0,193


In [121]:
alightings_df = od_matrix.filter(['route','busCode','tripNum','to_stop_id','end_time']) \
                        .groupby(['route','busCode','tripNum','to_stop_id']) \
                        .count() \
                        .reset_index() \
                        .rename(index=str, columns={'to_stop_id':'stop_id', 'end_time':'alighting_cnt'}) \
                        .sort_values('alighting_cnt', ascending=False)

In [122]:
alightings_df.head()

,route,busCode,tripNum,stop_id,alighting_cnt
32696,503,GE696,4.0,27538.0,358
32874,503,GR021,10.0,27538.0,229
40196,550,GE717,2.0,25906.0,178
32922,503,GR021,12.0,27538.0,170
34636,506,EL059,2.0,25899.0,127


In [123]:
passenger_flows = boardings_df.merge(alightings_df, how='outer') \
                                .sort_values(['boarding_cnt','alighting_cnt']) \
                                .assign(route = lambda x: x['route'].astype(str).str.replace("\.0",'').str.zfill(3),
                                        stop_id = lambda x: x['stop_id'].astype(int))

In [124]:
passenger_flows.head()

,route,busCode,tripNum,stop_id,boarding_cnt,alighting_cnt
22819,812,BA010,9.0,30306,1.0,1.0
22829,550,GE707,15.0,25906,1.0,1.0
22836,603,HD248,6.0,25396,1.0,1.0
22865,603,HD248,6.0,25395,1.0,1.0
22874,965,MN608,1.0,26346,1.0,1.0


In [125]:
vehicle_load = bus_trips.merge(passenger_flows, on=['route','busCode','tripNum','stop_id'], how='left') \
                        .assign(boarding_cnt = lambda x: x['boarding_cnt'].fillna(0),
                                alighting_cnt = lambda x: x['alighting_cnt'].fillna(0)) \
                        .assign(crowd_bal = lambda x: x['boarding_cnt'] - x['alighting_cnt']) \
                        .assign(num_pass = lambda x: x.groupby(['route','busCode','tripNum']) \
                                                        .crowd_bal.transform(pd.Series.cumsum))

In [126]:
vehicle_load.head()

,route,busCode,tripNum,stop_id,gps_datetime,stop_lat,stop_lon,parent_station,boarding_cnt,alighting_cnt,crowd_bal,num_pass
0,001,BN998,1.0,29887,2017-06-16 07:01:26,-25.428171,-49.264558,NaN,0.0,0.0,0.0,0.0
1,001,BN998,1.0,31453,2017-06-16 07:02:04,-25.430388,-49.263602,NaN,0.0,0.0,0.0,0.0
2,001,BN998,1.0,31454,2017-06-16 07:02:38,-25.433503,-49.262257,NaN,0.0,0.0,0.0,0.0
3,001,BN998,1.0,30748,2017-06-16 07:05:09,-25.435187,-49.264933,NaN,0.0,0.0,0.0,0.0
4,001,BN998,1.0,30749,2017-06-16 07:05:22,-25.435959,-49.266889,NaN,0.0,0.0,0.0,0.0


In [127]:
vehicle_load[(vehicle_load['busCode'] == 'HL309')].sort_values(['gps_datetime'])

,route,busCode,tripNum,stop_id,gps_datetime,stop_lat,stop_lon,parent_station,boarding_cnt,alighting_cnt,crowd_bal,num_pass
108637,204,HL309,1.0,26252,2017-06-16 05:29:56,-25.512983,-49.294505,14494.0,41.0,0.0,41.0,41.0
108638,204,HL309,1.0,26239,2017-06-16 05:36:56,-25.476439,-49.292220,14499.0,3.0,9.0,-6.0,35.0
108639,204,HL309,1.0,25719,2017-06-16 05:43:34,-25.449738,-49.284600,26147.0,2.0,8.0,-6.0,29.0
108640,204,HL309,1.0,25813,2017-06-16 05:48:22,-25.443506,-49.268244,26127.0,0.0,4.0,-4.0,25.0
108641,204,HL309,1.0,8727,2017-06-16 05:52:37,-25.433655,-49.263478,25427.0,1.0,8.0,-7.0,18.0
108642,204,HL309,1.0,25917,2017-06-16 05:57:00,-25.426995,-49.264729,26083.0,0.0,10.0,-10.0,8.0
108643,204,HL309,1.0,25712,2017-06-16 05:59:12,-25.419830,-49.268908,14567.0,0.0,5.0,-5.0,3.0
108644,204,HL309,1.0,26234,2017-06-16 06:04:37,-25.406600,-49.252188,14474.0,0.0,3.0,-3.0,0.0
108645,204,HL309,1.0,30447,2017-06-16 06:11:01,-25.393414,-49.240380,14471.0,0.0,0.0,0.0,0.0
108646,204,HL309,1.0,30454,2017-06-16 06:17:01,-25.377086,-49.224813,14470.0,20.0,0.0,20.0,20.0


#### Adjust vehicle load by a factor due to missed trips

In [ ]:
# Adjust number of passengers based on the trip loss inverse proportion for each vehicle/route during the processing phase. 

#### Some analyses

#### Most crowded stops

In [133]:
boarding_by_stop = vehicle_load.groupby('stop_id').boarding_cnt.sum().reset_index().sort_values('boarding_cnt', ascending=False)

In [134]:
boarding_by_stop.head()

,stop_id,boarding_cnt
788,27560,2892.0
446,26197,1818.0
217,25554,1601.0
2686,31633,1420.0
448,26199,1356.0


In [135]:
alighting_by_stop = vehicle_load.groupby('stop_id').alighting_cnt.sum().reset_index().sort_values('alighting_cnt', ascending=False)

In [136]:
alighting_by_stop.head()

,stop_id,alighting_cnt
593,26631,2596.0
489,26263,1787.0
448,26199,1515.0
144,25389,1461.0
878,28132,1441.0


In [141]:
stops_crowdedness = boarding_by_stop.merge(alighting_by_stop, on='stop_id') \
                                    .assign(stop_crowd = lambda x: x['boarding_cnt'] + x['alighting_cnt']) \
                                    .sort_values('stop_crowd', ascending=False)

In [142]:
stops_crowdedness.head()

,stop_id,boarding_cnt,alighting_cnt,stop_crowd
0,27560,2892.0,1184.0,4076.0
6,26263,1191.0,1787.0,2978.0
109,26631,328.0,2596.0,2924.0
4,26199,1356.0,1515.0,2871.0
1,26197,1818.0,945.0,2763.0


#### Most crowded routes

In [144]:
boardings_by_route = vehicle_load.groupby('route').boarding_cnt.sum().reset_index().sort_values('boarding_cnt', ascending=False)

In [145]:
boardings_by_route.head()

,route,boarding_cnt
28,203,10867.0
58,303,7702.0
11,050,6927.0
105,503,6835.0
10,040,6517.0
